# Курсовой проект по курсу "Python для Data Science"

*Студент*: **Starikova Anna**  
*Преподаватель*: Shirkin Sergey

*Материалы к проекту (файлы)*:  
train.csv  
test.csv

*Задание*:  
Используя данные из train.csv, построить модель для предсказания цен на недвижимость (квартиры). С помощью полученной модели предсказать цены для квартир из файла test.csv.

*Целевая переменная*:  
Price

*Основная метрика*:  
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

*Вспомогательная метрика*:  
MSE - средняя квадратичная ошибка (sklearn.metrics.mean_squared_error)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_columns = 100

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.metrics import r2_score as r2

### Загрузка и просмотр данных

In [2]:
data = pd.read_csv('D:/GeekBrains/Python for Data Science/Kurs_project_task/train.csv')

In [3]:
data.head(10)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644
5,12915,59,3.0,80.384479,46.683720,12.0,5,17.0,2011,0.309479,B,B,35,7715,4,990.0,0,6,B,215898.447742
6,14549,154,2.0,62.254114,37.160377,7.0,3,5.0,1960,0.460556,B,B,20,4386,14,NaN,1,5,B,296021.204377
7,11993,74,2.0,80.312926,NaN,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,221244.156664
8,5172,1,2.0,64.511437,NaN,1.0,9,17.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,229102.795999
9,8649,23,1.0,46.461409,18.915552,8.0,13,17.0,2014,0.075779,B,B,6,1437,3,NaN,0,2,B,95380.220993


In [4]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [6]:
data.shape

(10000, 20)

### Очистка и форматирование данных

#### Сначала оценим ситуацию с колонками, в которых имеются пропущенные данные

In [7]:
data.corr()['Healthcare_1'].sort_values()

HouseFloor      -0.143973
Floor           -0.138294
Ecology_1       -0.043547
Square          -0.039748
LifeSquare      -0.026867
HouseYear       -0.011969
Id              -0.003879
KitchenSquare    0.009472
Rooms            0.042857
Social_2         0.050458
Shops_1          0.057758
Social_1         0.077133
Social_3         0.133877
Price            0.138755
Helthcare_2      0.203177
DistrictId       0.304335
Healthcare_1     1.000000
Name: Healthcare_1, dtype: float64

In [8]:
#Видно, что существующие данные Healthcare_1 имеют слабую корреляцию как с целевым показателем Price, так и с другими признаками.
#Поэтому, учитывая что в Healthcare_1 много значений пропущенно, а корреляция с Price низкая, можно данный признак удалить.
data = data.drop(['Healthcare_1'], axis = 1)

In [9]:
data = data.drop(['LifeSquare'], axis = 1)

#### Теперь посмотрим остальные колонки

##### Rooms

In [10]:
#посмотрим на данные по квартирам с числом комнат более 5
data.loc[data['Rooms'] > 5, ['Id','Rooms', 'Square', 'KitchenSquare', 'HouseYear', 'Price']]

,Id,Rooms,Square,KitchenSquare,HouseYear,Price
377,5927,10.0,59.056975,10.0,2002,317265.323792
1454,8491,19.0,42.006046,7.0,2014,78364.616704
2170,14003,6.0,59.414334,6.0,1969,229661.964416
8849,14865,10.0,60.871266,10.0,1994,172329.270863


In [11]:
data.loc[data['Rooms'].isin([1,2,3,4,5]), ['Rooms', 'Price','Square']].groupby('Rooms').median()

,Price,Square
Rooms,,
1.0,154979.497993,40.406590
2.0,200320.754636,55.841812
3.0,268723.157677,77.413643
4.0,377524.231990,98.668054
5.0,455355.638212,116.084861


In [12]:
#заменим большие значения комнат на те, которые наиболее соответствуют по значению площади
data.loc[data['Id'].isin([5927, 14003, 14865]), 'Rooms'] = 2
data.loc[data['Id'] == 8491, 'Rooms'] = 1

In [13]:
#посмотрим на данные по квартирам с числом комнат 0
data.loc[data['Rooms'] < 1, ['Id','Rooms','Square','Price','KitchenSquare']].sort_values(['Square'])

,Id,Rooms,Square,Price,KitchenSquare
4853,3224,0.0,2.377248,126596.941798,0.0
6149,3159,0.0,38.697117,158998.110646,9.0
2269,7317,0.0,41.790881,98129.976788,0.0
3911,770,0.0,49.483501,217009.338463,0.0
4366,456,0.0,81.491446,212864.799112,0.0
8834,9443,0.0,87.762616,219281.918007,0.0
1397,12638,0.0,138.427694,268394.744389,0.0
1981,7917,0.0,212.932361,302211.260887,0.0


In [14]:
#заменим нулевые значения комнат на те, которые наиболее соответствуют по значению площади и/или цене
data.loc[data['Id'].isin([3224, 3159, 7317]), 'Rooms'] = 1
data.loc[data['Id'].isin([770]), 'Rooms'] = 2
data.loc[data['Id'].isin([456, 9443]), 'Rooms'] = 3
data.loc[data['Id'].isin([12638]), 'Rooms'] = 4
data.loc[data['Id'].isin([7917]), 'Rooms'] = 5

##### Square & KitchenSquare

In [15]:
#проверим значения, выбивающиеся "вверх"
data.loc[data['Square'] > 200, ['Id', 'Rooms', 'Square', 'KitchenSquare', 'Price']]

,Id,Rooms,Square,KitchenSquare,Price
1981,7917,5.0,212.932361,0.0,302211.260887
1982,5548,5.0,275.645284,26.0,455264.882666
4262,28,2.0,604.705972,1.0,187717.242538
4690,2307,1.0,409.425181,10.0,90470.430830
6977,11602,2.0,641.065193,10.0,133529.681562
9910,16568,4.0,200.334539,25.0,528560.506016


In [16]:
data.loc[data['Id'].isin([28, 2307, 11602]), ['Square']] = data.loc[data['Id'].isin([28, 2307, 11602]), ['Square', 'LifeSquare']] / 10
data.loc[data['Id'].isin([28, 2307, 11602])]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
4262,28,9,2.0,60.470597,1.0,17,18.0,1977,0.161532,B,B,25,5648,1,2,4,B,187717.242538
4690,2307,102,1.0,40.942518,10.0,4,4.0,2016,0.238617,B,B,26,3889,6,3,6,B,90470.430830
6977,11602,30,2.0,64.106519,10.0,20,19.0,2019,0.000078,B,B,22,6398,141,3,23,B,133529.681562


In [17]:
#проверим значения, выбивающиеся "вниз"
data.loc[data['Square'] < 15, ['Id', 'Rooms', 'Square', 'KitchenSquare', 'Price']].sort_values(['Price'])

,Id,Rooms,Square,KitchenSquare,Price
4739,12676,3.0,13.784865,7.0,78388.806186
3280,10527,1.0,4.380726,1.0,97560.720383
1316,11526,1.0,4.633498,1.0,107604.269441
4853,3224,1.0,2.377248,0.0,126596.941798
1608,10202,1.0,2.596351,1.0,137597.601458
4900,4504,3.0,4.390331,1.0,161379.067034
6392,14786,1.0,1.136859,1.0,181434.825589
8030,13265,3.0,4.823679,0.0,237716.681261
3413,9487,1.0,5.129222,1.0,369472.403061
212,1748,2.0,5.497061,1.0,412511.088764


In [18]:
#можно было бы умножить значение Square на 10, но все равно будет расхождение в комнатах и местами LifeSquare останется больше Square
#Суммарно Suqare < 15 имеет 13 записей, это немного от общего объема данных, можно удалить
data = data.drop(data.loc[data['Square'] < 15, :].index)
data.shape

(9987, 18)

In [19]:
#посмотрим на наличие данных, где площадь кухни превышают общую площадь
data[data['KitchenSquare'] > data['Square']]

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1064,14656,62,1.0,47.100719,2014.0,4,1.0,2014,0.072158,B,B,2,629,1,0,0,A,108337.484207
5149,13703,42,1.0,38.071692,73.0,9,10.0,2006,0.158249,B,B,21,5731,0,1,0,B,160488.033165
7088,6569,27,1.0,38.220258,84.0,4,17.0,2018,0.011654,B,B,4,915,0,0,0,B,99079.960518
8584,14679,81,1.0,32.276663,1970.0,6,1.0,1977,0.006076,B,B,30,5285,0,6,6,B,105539.556275


In [20]:
# заменим выбивающиеся значения на средние
kitchen_index = data[data['KitchenSquare'] > data['Square']].index
kitchen_index
data.loc[kitchen_index, 'KitchenSquare'] = round(data['KitchenSquare'].mean())

##### Floor & HouseFloor

In [21]:
#проверим значения, выбивающиеся "вверх"
data.loc[data['HouseFloor'] > 50]

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
6131,10806,5,1.0,51.944587,1.0,6,99.0,1977,0.150818,B,B,16,3433,4,4,5,B,296127.115515
8599,9300,74,2.0,71.747869,9.0,5,99.0,1977,0.075779,B,B,6,1437,3,0,2,B,243329.912579
8854,78,30,2.0,65.773749,1.0,8,117.0,1977,0.000078,B,B,22,6398,141,3,23,B,207248.370520


In [22]:
#for i in range(100):
#    print(i, data.loc[data['HouseYear']==1977, ['HouseFloor']].eq(i).sum() )
data.loc[data['HouseYear'] == 1977, ['HouseFloor']].describe()
#заменим значения 99 и 117 на более распространенные для 1977 года 9 и 17, соответственно
data.loc[data['HouseFloor'] == 99, 'HouseFloor'] = 9
data.loc[data['HouseFloor'] == 117, 'HouseFloor'] = 17

In [23]:
data.corr()['Price'][['Floor', 'HouseFloor']]

Floor         0.127220
HouseFloor    0.090572
Name: Price, dtype: float64

In [24]:
#Значение общего количества этажей в доме не может быть меньше, чем номер этажа для данной квартиры
#Поменяем местами этажность дома и номер этажа для всех Id, где они несоответвтуют 
floors_index = data[data['HouseFloor'] < data['Floor']].index
data.loc[floors_index, 'Floor'], data.loc[floors_index, 'HouseFloor'] = data.loc[floors_index, 'HouseFloor'], data.loc[floors_index, 'Floor']

In [25]:
data.corr()['Price'][['Floor', 'HouseFloor']]

Floor         0.110182
HouseFloor    0.115275
Name: Price, dtype: float64

In [26]:
data.head(20)

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,6.0,7.0,9.0,1969,0.089040,B,B,33,7976,5,0,11,B,184966.930730
1,15053,41,3.0,65.683640,8.0,7.0,9.0,1978,0.000070,B,B,46,10309,1,1,16,B,300009.450063
2,4765,53,2.0,44.947953,0.0,8.0,12.0,1968,0.049637,B,B,34,7759,0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,9.0,8.0,17.0,1977,0.437885,B,B,23,5735,3,0,5,B,175616.227217
4,10783,99,1.0,39.649192,7.0,11.0,12.0,1976,0.012339,B,B,35,5776,1,2,4,B,150226.531644
5,12915,59,3.0,80.384479,12.0,5.0,17.0,2011,0.309479,B,B,35,7715,4,0,6,B,215898.447742
6,14549,154,2.0,62.254114,7.0,3.0,5.0,1960,0.460556,B,B,20,4386,14,1,5,B,296021.204377
7,11993,74,2.0,80.312926,0.0,0.0,14.0,1977,0.075779,B,B,6,1437,3,0,2,B,221244.156664
8,5172,1,2.0,64.511437,1.0,9.0,17.0,1977,0.007122,B,B,1,264,0,0,1,B,229102.795999
9,8649,23,1.0,46.461409,8.0,13.0,17.0,2014,0.075779,B,B,6,1437,3,0,2,B,95380.220993


##### HouseYear

In [27]:
#проверим значения, выбивающиеся "вверх"
data.loc[data['HouseYear'] > 2020, :]

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1497,10814,109,1.0,37.265070,9.0,9.0,12.0,20052011,0.136330,B,B,30,6141,10,3,6,B,254084.534396
4189,11607,147,2.0,44.791836,5.0,4.0,9.0,4968,0.319809,B,B,25,4756,16,5,8,B,243028.603096


In [28]:
# вероятно 20052011 это года строительства 2005-2011, заменим на срок окончания, т.е. 2011
data.loc[data['HouseYear'] == 20052011, 'HouseYear'] = 2005

# 4968 поменяем на 1968
data.loc[data['HouseYear'] == 4968, 'HouseYear'] = 1968

In [29]:
# проверим значения, выбивающиеся "вниз" - отсутствуют
data.loc[data['HouseYear'] < 1910, :]

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price


#### Переведем категориальные признаки в числовой вид

In [30]:
def category_prep(df, cat_feats=['Ecology_2', 'Ecology_3', 'Shops_2']):
    df_copy = df.copy()
    for col in cat_feats:
        df_copy.loc[:, col] = df[col].map({'A': 1, 'B': 0})
    return df_copy

#### Переведем признаки, имеющие по факту целочисленные значения в int

In [31]:
def int_prep(df, cat_feats=['Rooms', 'KitchenSquare', 'Floor', 'HouseFloor']):
    df_copy = df.copy()
    for col in cat_feats:
        df_copy.loc[:, col] = df[col].astype(int)
    return df_copy

### Создание дополнительных характеристик

In [32]:
def add_mean_prices(df):
    df_out = df.copy()
    
    # средняя цена за квартиру по District и количеству комнат
    stats_flt_by_dist_rooms = df.groupby(['DistrictId', 'Rooms'], as_index=False)[['Price']]\
                                                                .mean().rename(columns={'Price':'mean_price_dist_rooms'})
    # средняя цена за квартиру по количеству комнат
    stats_flt_by_rooms = data.groupby(['Rooms'], as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price_rooms'})

    # средняя цена за квадратный метр по District
    data['Price_m2']= data['Price']/data['Square']
    stats_sqm_by_dist = data.groupby(['DistrictId'], as_index=False)[['Price_m2']]\
                                                                .mean().rename(columns={'Price_m2':'mean_price_m2'})
    mean_price = data['Price'].mean()
    
    df_out = pd.merge(df_out, stats_flt_by_dist_rooms, on=['DistrictId', 'Rooms'], how='left')
    df_out = pd.merge(df_out, stats_flt_by_rooms, on='Rooms', how='left')
    df_out = pd.merge(df_out, stats_sqm_by_dist, on='DistrictId', how='left')
    
    df_out['mean_price_rooms'] = df_out['mean_price_rooms'].fillna(mean_price)
    df_out['mean_price_m2'] = df_out['mean_price_m2'].fillna(df_out['mean_price_rooms'])
    df_out['mean_price_dist_rooms'] = df_out['mean_price_dist_rooms'].fillna(df_out['mean_price_rooms'])
   
    return df_out

### Создание необходимых функций

In [33]:
def prepare_data(df):
    df = category_prep(df)
    df = add_mean_prices(df)
    df = df.astype("float32")
    return df

### Подготовка сетов данных

In [34]:
train, valid = train_test_split(data, test_size=0.20, random_state=50)

In [35]:
train.shape, valid.shape

((7989, 18), (1998, 18))

### Обработка сетов

In [36]:
train = prepare_data(train)

In [37]:
valid = prepare_data(valid)

In [38]:
train.shape, valid.shape

((7989, 21), (1998, 21))

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7989 entries, 0 to 7988
Data columns (total 21 columns):
Id                       7989 non-null float32
DistrictId               7989 non-null float32
Rooms                    7989 non-null float32
Square                   7989 non-null float32
KitchenSquare            7989 non-null float32
Floor                    7989 non-null float32
HouseFloor               7989 non-null float32
HouseYear                7989 non-null float32
Ecology_1                7989 non-null float32
Ecology_2                7989 non-null float32
Ecology_3                7989 non-null float32
Social_1                 7989 non-null float32
Social_2                 7989 non-null float32
Social_3                 7989 non-null float32
Helthcare_2              7989 non-null float32
Shops_1                  7989 non-null float32
Shops_2                  7989 non-null float32
Price                    7989 non-null float32
mean_price_dist_rooms    7989 non-null float32


### Модель

In [40]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1998 entries, 0 to 1997
Data columns (total 21 columns):
Id                       1998 non-null float32
DistrictId               1998 non-null float32
Rooms                    1998 non-null float32
Square                   1998 non-null float32
KitchenSquare            1998 non-null float32
Floor                    1998 non-null float32
HouseFloor               1998 non-null float32
HouseYear                1998 non-null float32
Ecology_1                1998 non-null float32
Ecology_2                1998 non-null float32
Ecology_3                1998 non-null float32
Social_1                 1998 non-null float32
Social_2                 1998 non-null float32
Social_3                 1998 non-null float32
Helthcare_2              1998 non-null float32
Shops_1                  1998 non-null float32
Shops_2                  1998 non-null float32
Price                    1998 non-null float32
mean_price_dist_rooms    1998 non-null float32


In [41]:
train.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'KitchenSquare', 'Floor',
       'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2', 'Ecology_3',
       'Social_1', 'Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Shops_2',
       'Price', 'mean_price_dist_rooms', 'mean_price_rooms', 'mean_price_m2'],
      dtype='object')

In [42]:
train.describe()

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price,mean_price_dist_rooms,mean_price_rooms,mean_price_m2
count,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000
mean,8379.146484,50.610340,1.891100,56.387115,5.936538,7.739892,13.446614,1985.077637,0.121027,0.010014,0.029165,24.763424,5359.456055,7.963951,1.324446,4.225811,0.080736,214372.000000,214372.906250,214176.437500,3893.618164
std,4859.645020,43.635296,0.812939,19.328596,5.220896,5.319083,5.876493,18.476704,0.120298,0.099570,0.168282,17.494129,3991.784912,23.587372,1.497241,4.780397,0.272455,92850.398438,75071.125000,53588.378906,943.978455
min,0.000000,0.000000,1.000000,16.117155,0.000000,0.000000,1.000000,1910.000000,0.000000,0.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.777344,69623.750000,159543.078125,2284.108643
25%,4142.000000,19.000000,1.000000,41.790882,1.000000,4.000000,9.000000,1974.000000,0.019509,0.000000,0.000000,6.000000,1564.000000,0.000000,0.000000,1.000000,0.000000,153893.984375,166099.390625,159543.078125,3021.675049
50%,8391.000000,37.000000,2.000000,52.737148,6.000000,7.000000,14.000000,1977.000000,0.075779,0.000000,0.000000,25.000000,5285.000000,2.000000,1.000000,3.000000,0.000000,192589.812500,199989.984375,215224.890625,4084.464355
75%,12583.000000,75.000000,2.000000,65.980286,9.000000,11.000000,17.000000,2002.000000,0.195781,0.000000,0.000000,36.000000,7227.000000,5.000000,2.000000,6.000000,0.000000,249114.531250,251084.656250,215224.890625,4489.819336
max,16798.000000,208.000000,5.000000,275.645294,123.000000,42.000000,48.000000,2020.000000,0.521867,1.000000,1.000000,74.000000,19083.000000,141.000000,6.000000,23.000000,1.000000,633233.437500,608674.125000,404735.812500,9452.113281


In [43]:
feats = ['DistrictId', 'Rooms', 'Square', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3',
       'Helthcare_2', 'Shops_1', 'Shops_2']

In [44]:
train.loc[:, feats].describe()

,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
count,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000
mean,50.610340,1.891100,56.387115,5.936538,7.739892,13.446614,1985.077637,0.121027,0.010014,0.029165,24.763424,5359.456055,7.963951,1.324446,4.225811,0.080736
std,43.635296,0.812939,19.328596,5.220896,5.319083,5.876493,18.476704,0.120298,0.099570,0.168282,17.494129,3991.784912,23.587372,1.497241,4.780397,0.272455
min,0.000000,1.000000,16.117155,0.000000,0.000000,1.000000,1910.000000,0.000000,0.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,1.000000,41.790882,1.000000,4.000000,9.000000,1974.000000,0.019509,0.000000,0.000000,6.000000,1564.000000,0.000000,0.000000,1.000000,0.000000
50%,37.000000,2.000000,52.737148,6.000000,7.000000,14.000000,1977.000000,0.075779,0.000000,0.000000,25.000000,5285.000000,2.000000,1.000000,3.000000,0.000000
75%,75.000000,2.000000,65.980286,9.000000,11.000000,17.000000,2002.000000,0.195781,0.000000,0.000000,36.000000,7227.000000,5.000000,2.000000,6.000000,0.000000
max,208.000000,5.000000,275.645294,123.000000,42.000000,48.000000,2020.000000,0.521867,1.000000,1.000000,74.000000,19083.000000,141.000000,6.000000,23.000000,1.000000


In [45]:
model = RF(n_estimators=400,
    max_features=5,
    max_depth=50,
    random_state=50)

In [46]:
train.loc[:, feats].describe()

,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
count,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000,7989.000000
mean,50.610340,1.891100,56.387115,5.936538,7.739892,13.446614,1985.077637,0.121027,0.010014,0.029165,24.763424,5359.456055,7.963951,1.324446,4.225811,0.080736
std,43.635296,0.812939,19.328596,5.220896,5.319083,5.876493,18.476704,0.120298,0.099570,0.168282,17.494129,3991.784912,23.587372,1.497241,4.780397,0.272455
min,0.000000,1.000000,16.117155,0.000000,0.000000,1.000000,1910.000000,0.000000,0.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,1.000000,41.790882,1.000000,4.000000,9.000000,1974.000000,0.019509,0.000000,0.000000,6.000000,1564.000000,0.000000,0.000000,1.000000,0.000000
50%,37.000000,2.000000,52.737148,6.000000,7.000000,14.000000,1977.000000,0.075779,0.000000,0.000000,25.000000,5285.000000,2.000000,1.000000,3.000000,0.000000
75%,75.000000,2.000000,65.980286,9.000000,11.000000,17.000000,2002.000000,0.195781,0.000000,0.000000,36.000000,7227.000000,5.000000,2.000000,6.000000,0.000000
max,208.000000,5.000000,275.645294,123.000000,42.000000,48.000000,2020.000000,0.521867,1.000000,1.000000,74.000000,19083.000000,141.000000,6.000000,23.000000,1.000000


In [47]:
model.fit(train.loc[:, feats], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=400, n_jobs=None, oob_score=False, random_state=50,
           verbose=0, warm_start=False)

In [48]:
pred_train = model.predict(train.loc[:, feats])

In [49]:
pred_valid = model.predict(valid.loc[:, feats])

### Оценка

In [50]:
r2(train['Price'], pred_train)

0.9657729690011855

In [51]:
r2(valid['Price'], pred_valid)

0.7520605667103724

### Загрузка и обработка тестового набора

In [52]:
test = pd.read_csv('D:/GeekBrains/Python for Data Science/Kurs_project_task/test.csv')

In [53]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [54]:
test = test.drop(['Healthcare_1'], axis = 1)
test = test.drop(['LifeSquare'], axis = 1)
test.shape

(5000, 17)

In [55]:
test.loc[test['Rooms'] > 5, ['Id','Rooms', 'Square', 'KitchenSquare', 'HouseYear']]
test.shape

(5000, 17)

In [56]:
test.loc[test['Id'].isin([10793, 4058]), 'Rooms'] = 5
test.loc[test['Id'] == 1435, 'Rooms'] = 2

In [57]:
test.loc[test['Rooms'] < 1, ['Id','Rooms','Square', 'KitchenSquare']].sort_values(['Square'])

,Id,Rooms,Square,KitchenSquare
2524,10729,0.0,76.345154,12.0
2406,3343,0.0,116.824201,0.0


In [58]:
test.loc[test['Id']== 10729, 'Rooms'] = 3
test.loc[test['Id'] == 3343, 'Rooms'] = 5

In [59]:
test.loc[test['Square'] > 200, ['Id', 'Rooms', 'Square', 'KitchenSquare']]

,Id,Rooms,Square,KitchenSquare
3217,4058,5.0,223.453689,16.0
3909,1397,3.0,221.138768,4.0


In [60]:
test.loc[test['Square'] < 15, ['Id', 'Rooms', 'Square', 'KitchenSquare']]
sq_ind = test.loc[test['Square'] < 15, :].index
test.loc[sq_ind, 'Square'] = test.loc[sq_ind, 'Square']*10

In [61]:
test[test['KitchenSquare'] > test['Square']]

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
1964,14594,11,2.0,42.795304,620.0,11,14.0,1972,0.038693,B,B,28,6533,1,2,5,B
2424,10662,29,1.0,25.956829,33.0,5,5.0,1973,0.069660,B,B,31,6119,4,1,2,B
3863,3194,29,2.0,43.926987,44.0,6,5.0,1964,0.086885,B,B,10,2277,2,0,0,B
4555,12640,6,2.0,54.629142,97.0,4,17.0,2015,0.243205,B,B,5,1564,0,0,0,B


In [62]:
test.shape

(5000, 17)

In [63]:
kitchen_index = test[test['KitchenSquare'] > test['Square']].index
kitchen_index
test.loc[kitchen_index, 'KitchenSquare'] = round(test['KitchenSquare'].mean())

In [64]:
test.loc[test['HouseFloor'] > 50]

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
651,15864,27,3.0,47.722835,9.0,18,99.0,1977,0.072158,B,B,2,629,1,0,0,A


In [65]:
test.loc[test['HouseFloor'] == 99, 'HouseFloor'] = 19

In [66]:
test[test['HouseFloor'] < test['Floor']]

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
1,15856,74,2.0,69.263183,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,0,2,B
10,15975,61,3.0,64.543658,7.0,10,9.0,1981,0.300323,B,B,52,10311,6,1,9,B
14,5214,27,1.0,37.555197,1.0,5,1.0,1977,0.211401,B,B,9,1892,0,0,1,B
22,12694,27,3.0,83.670032,0.0,8,0.0,2015,0.072158,B,B,2,629,1,0,0,A
24,8968,27,2.0,69.849239,0.0,3,0.0,1977,0.011654,B,B,4,915,0,0,0,B
27,11160,58,1.0,48.610661,1.0,4,3.0,1977,0.437885,B,B,23,5735,3,0,5,B
30,2982,6,2.0,63.460684,0.0,13,0.0,1977,0.243205,B,B,5,1564,0,0,0,B
32,2449,1,1.0,66.426585,0.0,14,0.0,1977,0.007122,B,B,1,264,0,0,1,B
36,9348,205,1.0,37.480811,8.0,6,5.0,1933,0.169091,B,B,19,3856,10,2,5,B
41,12749,21,2.0,49.011976,5.0,6,5.0,1969,0.194489,B,B,47,8004,3,3,5,B


In [67]:
floors_index = test[test['HouseFloor'] < test['Floor']].index
test.loc[floors_index, 'Floor'], test.loc[floors_index, 'HouseFloor'] = test.loc[floors_index, 'HouseFloor'], test.loc[floors_index, 'Floor']

In [68]:
test.loc[test['HouseYear'] > 2020, :]

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2


In [69]:
test.loc[test['HouseYear'] < 1910, :]

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
2,5480,190,1.0,135.978185,12.0,2.0,5.0,1909,0.000000,B,B,30,7538,87,5,5,B
3253,12316,183,5.0,157.976101,12.0,7.0,8.0,1908,0.246624,B,B,14,2940,45,3,6,B


In [70]:
#test = category_prep(test)
test[["Ecology_2","Ecology_3","Shops_2"]] = (test[["Ecology_2","Ecology_3","Shops_2"]] == "A").astype(int)
test = test.astype("float32")
test.describe()

,Id,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000
mean,8412.607422,51.279202,1.908200,56.546494,5.822800,7.838000,13.379000,1984.392578,0.119874,0.009600,0.029800,24.933800,5406.886230,8.26260,1.319400,4.242800,0.082400
std,4832.676270,44.179382,0.809638,19.050095,4.633161,5.454978,6.113664,18.573030,0.120070,0.097516,0.170048,17.532305,4026.621338,23.86377,1.479935,4.777351,0.274992
min,1.000000,0.000000,1.000000,13.785426,0.000000,0.000000,1.000000,1908.000000,0.000000,0.000000,0.000000,0.000000,168.000000,0.00000,0.000000,0.000000,0.000000
25%,4221.750000,21.000000,1.000000,41.926032,1.000000,4.000000,9.000000,1973.000000,0.019509,0.000000,0.000000,6.000000,1564.000000,0.00000,0.000000,1.000000,0.000000
50%,8320.500000,37.000000,2.000000,52.933733,6.000000,7.000000,14.000000,1977.000000,0.072158,0.000000,0.000000,25.000000,5285.000000,2.00000,1.000000,3.000000,0.000000
75%,12598.250000,77.000000,2.000000,66.295633,9.000000,11.000000,17.000000,2000.000000,0.195781,0.000000,0.000000,36.000000,7287.000000,5.00000,2.000000,6.000000,0.000000
max,16795.000000,212.000000,5.000000,223.453690,112.000000,46.000000,78.000000,2020.000000,0.521867,1.000000,1.000000,74.000000,19083.000000,141.00000,6.000000,23.000000,1.000000


In [71]:
test['Price'] = model.predict(test.loc[:, feats])

In [72]:
test.loc[:, ['Id', 'Price']].to_csv('AStarikova_predictions.csv', index=None)

In [73]:
result = pd.read_csv('AStarikova_predictions.csv')

In [74]:
result.shape

(5000, 2)

In [75]:
result

,Id,Price
0,725.0,157793.040742
1,15856.0,222333.255625
2,5480.0,361321.327012
3,15664.0,328885.483359
4,14275.0,144890.224570
5,7633.0,207091.647187
6,13329.0,163153.065918
7,5502.0,221210.962559
8,4220.0,304315.344746
9,11538.0,211037.617109
